# Spike Challenge

*Objetivo* : Predecir precio de la leche a partir de variables climatológicas y macroeconómicas. 

## Libraries

En esta sección importamos las librerias que utilizaremos para nuestro analisis.

In [1]:
# pandas data frames
import pandas as pd

#datetime format
import datetime

#multidimensional arrays
import numpy as np

#visualizations
import seaborn as sns 
import matplotlib.pyplot as plt



## Data

In [2]:
varClim = pd.read_csv('precipitaciones.csv') #precipitaciones por region
varMacr = pd.read_csv('banco_central.csv') #variables macroeconomicas
precios  = pd.read_csv('precio_leche.csv') #variable dependiente

## Cleansing and Exploratory Data Analysis

En esta sección vamos a entender los datos que tenemos (distribución), limpiar inconsistencias, valores faltantes, valores extremos, entre otros. 

(496, 9)

In [21]:
varMacr.shape

(613, 85)

In [3]:
varClim.head()

,date,Coquimbo,Valparaiso,Metropolitana_de_Santiago,Libertador_Gral__Bernardo_O_Higgins,Maule,Biobio,La_Araucania,Los_Rios
0,1989-02-01,0.719033,0.117045,0.028272,0.0,0.724667,9.172945,22.595834,52.268010
1,1998-10-01,0.065216,0.000000,0.011493,0.0,0.008968,2.089674,14.702087,45.142041
2,2001-02-01,2.030026,0.047987,0.002634,0.0,0.032163,10.768351,29.465057,65.571098
3,2008-11-01,0.622784,0.000000,0.015872,0.0,1.734168,25.083557,64.901645,113.093488
4,2009-04-01,0.008065,0.000000,0.009396,0.0,12.080715,56.233424,96.364235,155.622235


In [14]:
varMacr.head()

,Periodo,Imacec_empalmado,Imacec_produccion_de_bienes,Imacec_minero,Imacec_industria,Imacec_resto_de_bienes,Imacec_comercio,Imacec_servicios,Imacec_a_costo_de_factores,Imacec_no_minero,...,Indice_de_tipo_de_cambio_real___TCR_promedio_1986_100,Indice_de_produccion_industrial,Indice_de_produccion_industrial__mineria,Indice_de_produccion_industrial_electricidad__gas_y_agua,Indice_de_produccion_industrial__manufacturera,Generacion_de_energia_electrica_CDEC_GWh,Indice_de_ventas_comercio_real_IVCM,Indice_de_ventas_comercio_real_no_durables_IVCM,Indice_de_ventas_comercio_real_durables_IVCM,Ventas_autos_nuevos
0,2013-03-01 00:00:00+00:00,102.796.218,10.546.824,978.554.365,102.297.537,114.994.919,110.729.395,100.064.328,103.163.651,103.405.852,...,865.903.468,102.761.705,98.915.705,NaN,108.387.837,580.480.672,NaN,NaN,NaN,28577.0
1,2013-04-01 00:00:00+00:00,101.664.842,999.272.757,927.333.293,104.485.589,102.199.311,106.098.291,102.600.107,101.937.319,102.766.884,...,867.806.065,968.087.179,914.276.663,NaN,104.545.429,544.815.032,NaN,NaN,NaN,32206.0
2,2013-05-01 00:00:00+00:00,101.642.954,993.959.922,96.133.164,105.445.361,968.789.055,100.462.117,104.083.216,101.953.299,102.322.796,...,878.030.947,978.471.874,966.913.278,NaN,99.468.801,56.891.916,NaN,NaN,NaN,31589.0
3,2013-06-01 00:00:00+00:00,998.310.201,968.367.884,102.400.933,999.212.106,89.190.493,939.504.414,103.857.161,100.222.381,995.139.235,...,913.180.348,966.647.135,981.820.974,NaN,94.969.678,56.594.867,NaN,NaN,NaN,28457.0
4,2013-07-01 00:00:00+00:00,964.696.194,961.051.418,980.029.022,100.882.112,901.785.886,92.776.599,969.309.272,962.069.134,962.804.305,...,909.263.447,100.100.749,990.198.382,NaN,102.128.977,586.329.899,NaN,NaN,NaN,31736.0


### Dates Standarization

In [4]:
varClim['date'].apply(pd.to_datetime)

0     1989-02-01
1     1998-10-01
2     2001-02-01
3     2008-11-01
4     2009-04-01
         ...    
491   1988-10-01
492   2019-08-01
493   2015-06-01
494   2016-09-01
495   1982-04-01
Name: date, Length: 496, dtype: datetime64[ns]

In [6]:
#varMacr['Periodo']  = pd.to_datetime(varMacr['Periodo'], errors = 'raise', dayfirst=False, yearfirst=True)

It seems like the monts are out of bounds. There is a month that seems to have  a 13 value. Since this row has no much information, we will drop it

In [7]:
varMacr.loc[varMacr['Periodo'] == '2020-13-01 00:00:00 UTC']


,Periodo,Imacec_empalmado,Imacec_produccion_de_bienes,Imacec_minero,Imacec_industria,Imacec_resto_de_bienes,Imacec_comercio,Imacec_servicios,Imacec_a_costo_de_factores,Imacec_no_minero,...,Indice_de_tipo_de_cambio_real___TCR_promedio_1986_100,Indice_de_produccion_industrial,Indice_de_produccion_industrial__mineria,Indice_de_produccion_industrial_electricidad__gas_y_agua,Indice_de_produccion_industrial__manufacturera,Generacion_de_energia_electrica_CDEC_GWh,Indice_de_ventas_comercio_real_IVCM,Indice_de_ventas_comercio_real_no_durables_IVCM,Indice_de_ventas_comercio_real_durables_IVCM,Ventas_autos_nuevos
89,2020-13-01 00:00:00 UTC,NaN,NaN,NaN,NaN,NaN,803.222.345,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,893.234.345,NaN


In [8]:
varMacr.shape

(614, 85)

In [9]:
varMacr = varMacr[varMacr['Periodo'] != '2020-13-01 00:00:00 UTC']

In [10]:
varMacr.shape


(613, 85)

In [13]:
varMacr['Periodo']  = pd.to_datetime(varMacr['Periodo'], errors = 'raise', dayfirst=False, yearfirst=True,utc = False)
varMacr['Periodo'].dt.tz_localize(None)

0     2013-03-01
1     2013-04-01
2     2013-05-01
3     2013-06-01
4     2013-07-01
         ...    
609   2008-08-01
610   2008-09-01
611   2008-10-01
612   2008-11-01
613   2008-12-01
Name: Periodo, Length: 613, dtype: datetime64[ns]

### Detecting Missing Values

In [48]:
varClim.fillna(np.nan,inplace = True)
varMacr.fillna(np.nan, inplace = True)

In [64]:
print('The total number of precipition columns is:', varClim.shape[1])

The total number of precipition columns is: 9


In [59]:
total = varClim.isnull().sum().sort_values(ascending=False)
percent = (varClim.isnull().sum()/varClim.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data

,Total,Percent
Los_Rios,0,0.0
La_Araucania,0,0.0
Biobio,0,0.0
Maule,0,0.0
Libertador_Gral__Bernardo_O_Higgins,0,0.0
Metropolitana_de_Santiago,0,0.0
Valparaiso,0,0.0
Coquimbo,0,0.0
date,0,0.0


In [65]:
print('The total number of macroeconomic varaibles is:', varMacr.shape[1])

The total number of macroeconomic varaibles is: 85


In [70]:
total = varMacr.isnull().sum().sort_values(ascending=False)
percent = (varMacr.isnull().sum()/varMacr.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[(missing_data.Percent >= 0.80)].count()

Total      56
Percent    56
dtype: int64

We have identified 56 rows out of 85 with more than 80% of missing data. That is a lot. 

### Data Distribution

## Visualization

In [42]:
def plot_timeSeries(data, x, y, title, xlabel , ylabel, dpi = 100):
    plt.figure(figsize=(16,5), dpi = dpi)
    plt.plot(x,y, color = 'tab:red')
    plt.gca().set(title, xlabel, ylabel)
    plt.show()

In [44]:
plot_timeSeries(varClim, x=varClim.index, y=varClim.value, title='Monthly anti-diabetic drug sales in Australia from 1992 to 2008.') 

AttributeError: 'DataFrame' object has no attribute 'value'